# Modeling Dataset Preparation

**Purpose:** Create a clean feature matrix for logistic regression

**Goal:** Build `df_modeling_clean` with all characteristics and no nulls

**Output:** Ready-to-use dataset for predictive modeling with 20+ features

## Step 0: Setup

In [1]:
import polars as pl
import numpy as np
import pandas as pd
from google.cloud import bigquery
from scipy import stats as scipy_stats
from pathlib import Path

# Initialize BigQuery client
client = bigquery.Client()

# Create output directory
output_dir = Path('outputs')
output_dir.mkdir(exist_ok=True)

print("✅ Setup complete")

✅ Setup complete


## Step 1: Load Base Prescriber Revenue Data

In [4]:
# Query: Basic prescriber metrics from rx_claims
query_base = """
SELECT 
  PRESCRIBER_NPI_NBR,
  COUNT(*) as prescription_count,
  COUNT(DISTINCT NDC_DRUG_NM) as unique_drugs,
  SUM(TOTAL_PAID_AMT) as total_revenue,
  AVG(TOTAL_PAID_AMT) as avg_claim_amount,
  
  -- Brand preference
  COUNTIF(NDC_PREFERRED_BRAND_NM IS NOT NULL AND NDC_PREFERRED_BRAND_NM != '') as brand_count,
  
  -- Payer mix
  COUNTIF(PAYER_PLAN_CHANNEL_NM = 'Commercial') as commercial_count,
  COUNTIF(PAYER_PLAN_CHANNEL_NM = 'Medicare') as medicare_count,
  COUNTIF(PAYER_PLAN_CHANNEL_NM = 'Medicaid') as medicaid_count
  
FROM 
  `unique-bonbon-472921-q8.Claims.rx_claims`
WHERE 
  PRESCRIBER_NPI_NBR IS NOT NULL
  AND TOTAL_PAID_AMT IS NOT NULL
  AND TOTAL_PAID_AMT > 0
GROUP BY 
  PRESCRIBER_NPI_NBR
HAVING 
  prescription_count >= 10
"""

print("Executing base query...")
df_base = pl.from_arrow(client.query(query_base).to_arrow())

# Calculate derived features
df_base = df_base.with_columns([
    (pl.col('total_revenue') / pl.col('prescription_count')).alias('revenue_per_rx'),
    (pl.col('brand_count') / pl.col('prescription_count')).alias('brand_rate'),
    (pl.col('commercial_count') / pl.col('prescription_count')).alias('commercial_rate'),
    (pl.col('medicare_count') / pl.col('prescription_count')).alias('medicare_rate'),
    (pl.col('medicaid_count') / pl.col('prescription_count')).alias('medicaid_rate')
])

# Create target variable: top 10% by revenue
revenue_90th = df_base['total_revenue'].quantile(0.90)
df_base = df_base.with_columns([
    (pl.col('total_revenue') >= revenue_90th).cast(pl.Int32).alias('is_high_value')
])
print(df_base)
print(f"✅ Loaded {len(df_base):,} prescribers")
print(f"   Revenue threshold (90th percentile): ${revenue_90th:,.0f}")
print(f"   High-value prescribers: {df_base.filter(pl.col('is_high_value') == 1).height:,}")

Executing base query...
shape: (61_091, 15)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ PRESCRIBE ┆ prescript ┆ unique_dr ┆ total_rev ┆ … ┆ commercia ┆ medicare_ ┆ medicaid_ ┆ is_high_ │
│ R_NPI_NBR ┆ ion_count ┆ ugs       ┆ enue      ┆   ┆ l_rate    ┆ rate      ┆ rate      ┆ value    │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ i64       ┆ i64       ┆ i64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ i32      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 147771481 ┆ 521       ┆ 9         ┆ 3.7869e6  ┆ … ┆ 0.452975  ┆ 0.318618  ┆ 0.103647  ┆ 1        │
│ 4         ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 151833059 ┆ 126       ┆ 6         ┆ 1.1769e6  ┆ … ┆ 0.230159  ┆ 0.095238  ┆ 0.52381   ┆ 0        │
│ 6         ┆           ┆           ┆          

## Step 2: Add Refill Patterns (90-day supply)

In [5]:
query_refill = """
SELECT 
  PRESCRIBER_NPI_NBR,
  COUNTIF(DAYS_SUPPLY_VAL >= 90) as supply_90day_count,
  COUNT(*) as total_prescriptions,
  AVG(DAYS_SUPPLY_VAL) as avg_days_supply
FROM 
  `unique-bonbon-472921-q8.Claims.rx_claims`
WHERE 
  PRESCRIBER_NPI_NBR IS NOT NULL
  AND DAYS_SUPPLY_VAL IS NOT NULL
  AND DAYS_SUPPLY_VAL > 0
  AND DAYS_SUPPLY_VAL <= 365
GROUP BY 
  PRESCRIBER_NPI_NBR
HAVING 
  total_prescriptions >= 10
"""

print("Loading refill patterns...")
df_refill = pl.from_arrow(client.query(query_refill).to_arrow())

df_refill = df_refill.with_columns([
    (pl.col('supply_90day_count') / pl.col('total_prescriptions')).alias('supply_90day_rate')
])

# Join with base
df_base = df_base.join(
    df_refill.select(['PRESCRIBER_NPI_NBR', 'supply_90day_rate', 'avg_days_supply']),
    on='PRESCRIBER_NPI_NBR',
    how='left'
)
print(df_base)
print(f"✅ Added refill patterns for {df_refill.height:,} prescribers")

Loading refill patterns...
shape: (61_091, 17)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ PRESCRIBE ┆ prescript ┆ unique_dr ┆ total_rev ┆ … ┆ medicaid_ ┆ is_high_v ┆ supply_90 ┆ avg_days │
│ R_NPI_NBR ┆ ion_count ┆ ugs       ┆ enue      ┆   ┆ rate      ┆ alue      ┆ day_rate  ┆ _supply  │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ i64       ┆ i64       ┆ i64       ┆ f64       ┆   ┆ f64       ┆ i32       ┆ f64       ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 147771481 ┆ 521       ┆ 9         ┆ 3.7869e6  ┆ … ┆ 0.103647  ┆ 1         ┆ 0.059014  ┆ 40.46887 │
│ 4         ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 2        │
│ 151833059 ┆ 126       ┆ 6         ┆ 1.1769e6  ┆ … ┆ 0.52381   ┆ 0         ┆ 0.0       ┆ 30.80810 │
│ 6         ┆           ┆           ┆       

## Step 3: Add Prescribing Velocity (Growth Rate)

In [6]:
query_velocity = """
SELECT 
  PRESCRIBER_NPI_NBR,
  FORMAT_DATE('%Y-%m', SERVICE_DATE_DD) as month,
  COUNT(*) as monthly_rx_count
FROM 
  `unique-bonbon-472921-q8.Claims.rx_claims`
WHERE 
  PRESCRIBER_NPI_NBR IS NOT NULL
  AND SERVICE_DATE_DD IS NOT NULL
GROUP BY 
  PRESCRIBER_NPI_NBR, month
"""

print("Loading monthly prescription data...")
df_velocity = pl.from_arrow(client.query(query_velocity).to_arrow())

# Calculate growth rate using pandas + scipy
def fit_slope(group):
    """Calculate linear regression slope"""
    if len(group) < 3:
        return None
    x = np.arange(len(group))
    y = group['monthly_rx_count'].values
    slope, _, _, _, _ = scipy_stats.linregress(x, y)
    return slope

df_velocity_pd = df_velocity.to_pandas().sort_values(['PRESCRIBER_NPI_NBR', 'month'])
velocity_metrics_pd = df_velocity_pd.groupby('PRESCRIBER_NPI_NBR').apply(
    lambda g: pd.Series({
        'growth_rate_monthly': fit_slope(g),
        'months_observed': len(g)
    })
).reset_index()

df_velocity_metrics = pl.from_pandas(velocity_metrics_pd).drop_nulls()

# Cast NPI to match
df_velocity_metrics = df_velocity_metrics.with_columns([
    pl.col('PRESCRIBER_NPI_NBR').cast(pl.Int64)
])

# Join with base
df_base = df_base.join(
    df_velocity_metrics.select(['PRESCRIBER_NPI_NBR', 'growth_rate_monthly']),
    on='PRESCRIBER_NPI_NBR',
    how='left'
)

print(f"✅ Added growth rates for {df_velocity_metrics.height:,} prescribers")

Loading monthly prescription data...
✅ Added growth rates for 121,380 prescribers


/var/folders/89/tbzc8z7n42l5h22ynj6019y80000gn/T/ipykernel_57993/3495148900.py:29: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  velocity_metrics_pd = df_velocity_pd.groupby('PRESCRIBER_NPI_NBR').apply(


## Step 4: Add Condition Complexity (from medical_claims)

In [7]:
query_conditions = """
SELECT 
  CAST(PRIMARY_HCP AS INT64) as PRESCRIBER_NPI_NBR,
  COUNT(DISTINCT condition_label) as unique_conditions,
  COUNT(*) as total_med_claims
FROM 
  `unique-bonbon-472921-q8.Claims.medical_claims`
WHERE 
  PRIMARY_HCP IS NOT NULL
  AND condition_label IS NOT NULL
  AND condition_label != ''
GROUP BY 
  PRESCRIBER_NPI_NBR
HAVING 
  total_med_claims >= 5
"""

print("Loading condition complexity...")
df_conditions = pl.from_arrow(client.query(query_conditions).to_arrow())

# Join with base
df_base = df_base.join(
    df_conditions.select(['PRESCRIBER_NPI_NBR', 'unique_conditions']),
    on='PRESCRIBER_NPI_NBR',
    how='left'
)

print(f"✅ Added condition data for {df_conditions.height:,} prescribers")

Loading condition complexity...
✅ Added condition data for 748,048 prescribers


## Step 5: Add Pharma Payment Metrics

**Why these features:**
1. **Payment receipt** - Section 3.1 shows 90% of top 10% receive payments vs 81% bottom 90%
2. **Total payment amount** - Correlation with prescribing behavior
3. **Payment-product alignment** - 24.6% vs 13.9% alignment (Cohen's d = 0.36, MEDIUM effect)

In [8]:
query_payments = """
SELECT 
  CAST(npi_number AS INT64) as PRESCRIBER_NPI_NBR,
  COUNT(*) as payment_count,
  SUM(total_payment_amount) as total_payments,
  COUNT(DISTINCT associated_product) as unique_products_paid
FROM 
  `unique-bonbon-472921-q8.HCP.provider_payments`
WHERE 
  npi_number IS NOT NULL
  AND total_payment_amount > 0
GROUP BY 
  PRESCRIBER_NPI_NBR
"""

print("Loading pharma payment data...")
df_payments = pl.from_arrow(client.query(query_payments).to_arrow())

# Join with base
df_base = df_base.join(
    df_payments,
    on='PRESCRIBER_NPI_NBR',
    how='left'
)

# Create binary feature: receives payments or not
df_base = df_base.with_columns([
    pl.when(pl.col('payment_count').is_not_null())
      .then(1)
      .otherwise(0)
      .alias('receives_payments'),
    # Fill nulls with 0 for payment amounts
    pl.col('total_payments').fill_null(0),
    pl.col('payment_count').fill_null(0),
    pl.col('unique_products_paid').fill_null(0)
])
print(df_base)
print(f"✅ Added payment data for {df_payments.height:,} prescribers")

Loading pharma payment data...
shape: (61_091, 23)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ PRESCRIBE ┆ prescript ┆ unique_dr ┆ total_rev ┆ … ┆ payment_c ┆ total_pay ┆ unique_pr ┆ receives │
│ R_NPI_NBR ┆ ion_count ┆ ugs       ┆ enue      ┆   ┆ ount      ┆ ments     ┆ oducts_pa ┆ _payment │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ id        ┆ s        │
│ i64       ┆ i64       ┆ i64       ┆ f64       ┆   ┆ i64       ┆ f64       ┆ ---       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆ i64       ┆ i32      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 147771481 ┆ 521       ┆ 9         ┆ 3.7869e6  ┆ … ┆ 240       ┆ 5294.12   ┆ 32        ┆ 1        │
│ 4         ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 151833059 ┆ 126       ┆ 6         ┆ 1.

## Step 6: Add Procedure Involvement (from medical_claims)

In [9]:
query_procedures = """
SELECT 
  CAST(PRIMARY_HCP AS INT64) as PRESCRIBER_NPI_NBR,
  COUNT(*) as total_claims,
  COUNTIF(PROCEDURE_CD IS NOT NULL AND PROCEDURE_CD != '') as procedure_claims,
  COUNT(DISTINCT PROCEDURE_CD) as unique_procedures
FROM 
  `unique-bonbon-472921-q8.Claims.medical_claims`
WHERE 
  PRIMARY_HCP IS NOT NULL
GROUP BY 
  PRESCRIBER_NPI_NBR
HAVING 
  total_claims >= 5
"""

print("Loading procedure involvement...")
df_procedures = pl.from_arrow(client.query(query_procedures).to_arrow())

df_procedures = df_procedures.with_columns([
    (pl.col('procedure_claims') / pl.col('total_claims')).alias('procedure_rate')
])

# Join with base
df_base = df_base.join(
    df_procedures.select(['PRESCRIBER_NPI_NBR', 'procedure_rate', 'unique_procedures']),
    on='PRESCRIBER_NPI_NBR',
    how='left'
)
print(df_base)
print(f"✅ Added procedure data for {df_procedures.height:,} prescribers")

Loading procedure involvement...
shape: (61_091, 25)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ PRESCRIBE ┆ prescript ┆ unique_dr ┆ total_rev ┆ … ┆ unique_pr ┆ receives_ ┆ procedure ┆ unique_p │
│ R_NPI_NBR ┆ ion_count ┆ ugs       ┆ enue      ┆   ┆ oducts_pa ┆ payments  ┆ _rate     ┆ rocedure │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ id        ┆ ---       ┆ ---       ┆ s        │
│ i64       ┆ i64       ┆ i64       ┆ f64       ┆   ┆ ---       ┆ i32       ┆ f64       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆ i64       ┆           ┆           ┆ i64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 147771481 ┆ 521       ┆ 9         ┆ 3.7869e6  ┆ … ┆ 32        ┆ 1         ┆ 0.959128  ┆ 51       │
│ 4         ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ 151833059 ┆ 126       ┆ 6         ┆ 

## Step 7: Add Provider Biographical Data

**Why this feature:**
- Section 4.2 mentions credential patterns differentiate high-value prescribers
- Certifications, education, awards indicate opinion leader status
- KOL potential correlates with prescribing influence

In [10]:
query_bio = """
SELECT 
  CAST(npi_number AS INT64) as PRESCRIBER_NPI_NBR,
  ARRAY_LENGTH(certifications) as certification_count,
  ARRAY_LENGTH(education) as education_count,
  ARRAY_LENGTH(awards) as award_count,
  ARRAY_LENGTH(memberships) as membership_count
FROM 
  `unique-bonbon-472921-q8.HCP.providers_bio`
WHERE 
  npi_number IS NOT NULL
"""

print("Loading provider biographical data...")
df_bio = pl.from_arrow(client.query(query_bio).to_arrow())

# Calculate total credentials
df_bio = df_bio.with_columns([
    (pl.col('certification_count').fill_null(0) + 
     pl.col('education_count').fill_null(0) + 
     pl.col('award_count').fill_null(0) + 
     pl.col('membership_count').fill_null(0)).alias('total_credentials')
])

# Join with base
df_base = df_base.join(
    df_bio.select(['PRESCRIBER_NPI_NBR', 'certification_count', 'total_credentials']),
    on='PRESCRIBER_NPI_NBR',
    how='left'
)

# Fill nulls with 0 for credential counts
df_base = df_base.with_columns([
    pl.col('certification_count').fill_null(0),
    pl.col('total_credentials').fill_null(0)
])

print(f"✅ Added biographical data for {df_bio.height:,} prescribers")

Loading provider biographical data...
✅ Added biographical data for 819,282 prescribers


## Step 8: Create Clean Modeling Dataset

In [12]:
# Define feature columns for modeling
feature_columns = [
    # Basic metrics (Section 1)
    'total_revenue',
    'prescription_count',
    'unique_drugs',
    'avg_claim_amount',
    'revenue_per_rx',
    
    # Portfolio (Section 2)
    'brand_rate',
    
    # Payer mix (Section 5)
    'commercial_rate',
    'medicare_rate',
    'medicaid_rate',
    
    # Pharma payments (Section 3)
    'receives_payments',
    'total_payments',
    'payment_count',
    'unique_products_paid',
    
    # Advanced characteristics (Section 6)
    'growth_rate_monthly',
    'supply_90day_rate',
    'avg_days_supply',
    'unique_conditions',
    'procedure_rate',
    'unique_procedures',
    
    # Biographical (Section 4)
    'certification_count',
    'total_credentials'
]

# Select target + features and drop nulls
df_modeling_clean = df_base.select(['is_high_value'] + feature_columns).drop_nulls()

print("\n" + "="*80)
print("MODELING DATASET SUMMARY")
print("="*80)
print(f"\nTotal samples: {len(df_modeling_clean):,}")
print(f"High-value (class 1): {df_modeling_clean.filter(pl.col('is_high_value') == 1).height:,}")
print(f"Low-value (class 0): {df_modeling_clean.filter(pl.col('is_high_value') == 0).height:,}")
print(f"\nFeatures ({len(feature_columns)}):")
for i, col in enumerate(feature_columns, 1):
    print(f"  {i:2d}. {col}")

# Show sample
print("\nSample rows:")
print(df_modeling_clean.head(5))

# Check for any remaining nulls
null_counts = df_modeling_clean.null_count()
print("\nNull counts per column:")
print(null_counts)


MODELING DATASET SUMMARY

Total samples: 53,643
High-value (class 1): 5,933
Low-value (class 0): 47,710

Features (21):
   1. total_revenue
   2. prescription_count
   3. unique_drugs
   4. avg_claim_amount
   5. revenue_per_rx
   6. brand_rate
   7. commercial_rate
   8. medicare_rate
   9. medicaid_rate
  10. receives_payments
  11. total_payments
  12. payment_count
  13. unique_products_paid
  14. growth_rate_monthly
  15. supply_90day_rate
  16. avg_days_supply
  17. unique_conditions
  18. procedure_rate
  19. unique_procedures
  20. certification_count
  21. total_credentials

Sample rows:
shape: (5, 22)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ is_high_v ┆ total_rev ┆ prescript ┆ unique_dr ┆ … ┆ procedure ┆ unique_pr ┆ certifica ┆ total_cr │
│ alue      ┆ enue      ┆ ion_count ┆ ugs       ┆   ┆ _rate     ┆ ocedures  ┆ tion_coun ┆ edential │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆

## Step 9: Export for Use in Main Notebook

In [13]:
# Export to CSV for reference
output_path = output_dir / 'modeling_dataset_clean.csv'
df_modeling_clean.write_csv(output_path)

print(f"✅ Exported modeling dataset to: {output_path}")
print(f"\n📊 Dataset is ready for logistic regression modeling")
print(f"   Use df_modeling_clean in your analysis")
print(f"\n🎉 Total features: {len(feature_columns)}")

✅ Exported modeling dataset to: outputs/modeling_dataset_clean.csv

📊 Dataset is ready for logistic regression modeling
   Use df_modeling_clean in your analysis

🎉 Total features: 21
